# This code geocodes the intersections(i.e. gets the latitude and longitude corresponding to the street address) based on the On-Street Name and Cross-Street Name available to us and then Reverse Geocodes(i.e. gets address corresponding to latitude and longitude values) based on the goecoded lat-long to get the Borough names.
## As there is a limit on the number of locations one can geocode and reverse geocode in a day using the pygeocoder package which is a Python interface for the Google Geocoding API V3, we only geocode the locations corresponding to fatalities

In [ ]:
# Load packages
from pygeocoder import Geocoder
import pandas as pd
import numpy as np

Read the file which contains only the missing lat-long values for the intersections as a dataframe

In [ ]:
df = pd.read_csv('BOOK1.csv')

This next cell Geocodes the data for us.
    We use the time package to have a pause in between each iteration of the geocoding process as without this we 
    get and "OVER_QUERY_LIMIT" which possibly means that we tried to geocode too many locations one after the other

In [ ]:
import time
def geocoding():
    for index, row in df.iterrows():
        result=Geocoder.geocode(str(row['ON STREET NAME']).strip() + ' AND ' + str(row['CROSS STREET NAME']).strip() + ', NY')
        #The previous line joins the street names together inorder to geocode it as an address
        df.set_value(index, 'LATITUDE', float(result.coordinates[0]))
        df.set_value(index, 'LONGITUDE', float(result.coordinates[1]))
        time.sleep( 1 )


We call the function geocoding() in the next cell

In [ ]:
geocoding()

The next cell joins the latitude and longitudes obtained via goecoding and adds them in the 'LOCATION' column of the dataframe as a pd.core.series.Series type

In [ ]:
for index, row in df.iterrows():
    df.set_value(index, 'LOCATION', ('(' + str(row['LATITUDE']) + ', ' + str(row['LONGITUDE']) + ')'))
df['LOCATION']=df['LOCATION'].astype(pd.core.series.Series)

The next cell reverse geocodes based on the latitude and longitude values obtained by geocoding the intersections and gets the corresponding Borough names using the sublocality parameter

In [ ]:
df['BOROUGH']=df['BOROUGH'].astype(str)
import time
def reverse_geocoding():
    for index, row in df.iterrows():
        df.set_value(index, 'BOROUGH', str(Geocoder.reverse_geocode(row['LATITUDE'], row['LONGITUDE']).sublocality.upper()))
        time.sleep( 1 )

We call the function reverse_geocoding() in the next cell

In [ ]:
reverse_geocoding()

This may throw an error at some lat-long values stating that the lat-long value could not be reverse geocoded. In such a case we may need to manually check the Borough corresponding to that lat-long value using Google Maps or some other similar software and set it accordingly as shown in next cell

In [ ]:
df.set_value(88, 'BOROUGH', 'BROOKLYN')

Read the file containing all but the missing lat-long values as another dataframe

In [ ]:
df2 = pd.read_csv('BOOK2.csv')

Join the two dataframes together

In [ ]:
join = [df2, df]

final_df = pd.concat(join)

Save the final dataframe as a .csv file

In [ ]:
final_df.to_csv('SUPER_CLEANED_NYPD_TRAFFIC_COLLISION.csv')